In [ ]:
using Polly;
using Polly.Retry;
using Polly.CircuitBreaker;
using System;
using System.Net;
using System.Threading.Tasks;
using RestSharp;

public class ResiliencePolicyExample
{
    // Define the Retry policy
    private static readonly AsyncRetryPolicy<RestResponse> RetryPolicy = Policy
        .HandleResult<RestResponse>(resp => 
            resp.StatusCode == HttpStatusCode.TooManyRequests || 
            (int)resp.StatusCode >= 500
        )
        .WaitAndRetryAsync(
            retryCount: 4, 
            sleepDurationProvider: retryAttempt => TimeSpan.FromSeconds(5 + retryAttempt),
            onRetry: (response, timespan, retryCount, context) =>
            {
                Console.WriteLine($"Retry {retryCount} after {timespan.TotalSeconds} seconds due to: {response.Result.StatusCode}");
            }
        );

    // Define the Circuit Breaker policy
    private static readonly AsyncCircuitBreakerPolicy<RestResponse> CircuitBreakerPolicy = Policy
        .HandleResult<RestResponse>(res => (int)res.StatusCode >= 500)
        .CircuitBreakerAsync(
            exceptionsAllowedBeforeBreaking: 4, 
            durationOfBreak: TimeSpan.FromMinutes(1)
        );

    // Combine Retry and Circuit Breaker policies
    private static readonly PolicyWrap<RestResponse> CombinedPolicy = Policy.WrapAsync(RetryPolicy, CircuitBreakerPolicy);

    public static async Task RunPoliciesAsync()
    {
        var client = new RestClient("https://api.example.com/resource");
        var request = new RestRequest();

        try
        {
            var response = await CombinedPolicy.ExecuteAsync(async () =>
            {
                var result = await client.ExecuteAsync(request);
                return result;
            });

            Console.WriteLine($"Final response status code: {response.StatusCode}");
        }
        catch (BrokenCircuitException ex)
        {
            Console.WriteLine($"Circuit breaker is open: {ex.Message}");
        }
        catch (Exception ex)
        {
            Console.WriteLine($"Operation failed: {ex.Message}");
        }
    }
}

await ResiliencePolicyExample.RunPoliciesAsync();
